<a href="https://colab.research.google.com/github/qwertyrc/hello-world/blob/master/AI_Hols_de_Geier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ハゲタカのえじき　強化学習
ハゲタカのえじきに対して強化学習をして最強の戦法を見つけ出そう！



## ハゲタカのえじき(Gameクラス)

In [0]:
import random
import collections

class Game:
    def __init__(self):
        self.turn_count = 0
        self.deck = Deck()
        self.a = DQN_LearningPlayer("A")
        self.b = Player("B")
        self.c = Player("C")
        self.d = Player("D")
        self.players = [self.a,self.b,self.c,self.d]
        self.skip_score = 0 #引き分けによる持ち越しスコア
        self.deck.show()
        self.winner_name = ""
    
    def get_observation(self):
        observation = [self.deck.deck, self.deck.brd, self.a.hands.hands,
                      self.b.hands.hands, self.c.hands.hands, self.d.hands.hands]
        return observation
    
    
    
    def game(self):
        for i in range(15):
            self.turn_count = i
            #print("\n[Turn:", i+1, "]")
            g.turn()
        
        winner = max(self.players, key=lambda x:x.score)
        return winner

    def turn(self):
        self.deck.print()
        for p in self.players:
            p.play(self)
        self.judge()
        self.show_score()
        self.deck.show()
    
    def step(self, input):
        if self.a.play(self,input) == False:
            reward = -1
            observation = [self.deck.deck, self.deck.brd, self.a.hands.hands,
                          self.b.hands.hands, self.c.hands.hands, self.d.hands.hands]
            done = True
            return reward,observation,done
        self.b.play(self)
        self.c.play(self)
        self.d.play(self)
        self.judge()
        self.turn_count += 1
        
        reward = 0
        observation = [self.deck.deck, self.deck.brd, self.a.hands.hands,
                      self.b.hands.hands, self.c.hands.hands, self.d.hands.hands]
        done = False
        if self.turn_count == 15:
            done = True
            top_player = max(self.players, key=lambda x:x.score)
            worst_player = min(self.players, key=lambda x:x.score)
            if top_player == self.a:
                reward = 1
            elif worst_player == self.a:
                reward = 0
            else:
                reward = 0
            #reward = self.a.score
            return reward,observation,done
        self.deck.show()
        
        return reward,observation,done
        
        
    def judge(self):
        board = self.deck.board()
        cards = []
        for p in self.players:
            cards.append(p.show())
        
        if board > 0:
            cards.sort(reverse=True)
        else:
            cards.sort()
        
        win_card=0
        count = collections.Counter(cards)
        for card in cards:
            if count[card] == 1:
                win_card = card
                break
        
        if win_card == 0: #no winner
            self.skip_score += board
            #print("Draw")
            self.winner_name = "Draw"
            return

        for p in self.players:
            if p.show() == win_card:
                p.score += board + self.skip_score
                self.skip_score = 0
                #print("Win:",p.name)
                self.winner_name = p.name
                break
    
    def show_score(self):
        print("---score---")
        for p in self.players:
            print(p.name,":",p.score)
        

class Deck:
    def __init__(self):
        self.deck = [-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
    
    def show(self):
        if self.deck != []:
            self.brd = random.choice(self.deck)
            self.deck.remove(self.brd)
        return self.brd
    
    def board(self):
        return self.brd

    def print(self):
        #print("board:",self.brd)
        pass

#完全ランダムCPU
class Player:
    def __init__(self,name):
        self.hands = Hands()
        self.name = name
        self.score = 0
        self.choice = 0
        
    def play(self,game):
        self.choice = random.choice(self.hands.show())
        self.hands.play(self.choice)
        #print(self.name, " plays ", self.choice)

    def show(self):
        return self.choice
    
    def print_choice(self):
        print(self.name, " plays ", self.choice)

#手入力
class Human(Player):
    def play(self,game):
        self.hands.print()
        while True:
            data = 0
            while True:
                try:
                    input_data = input(self.name+" plays >> ")
                    data = int(input_data)
                    break
                except ValueError:
                    print("not number")
            self.choice = data
            if self.hands.play(self.choice):
                break

#小さいものは小さい数字，大きいものは大きい数字
class LinearCPU(Player):
    def play(self,game):
        d = {-5: 1, -4:2, -3:3, -2:4, -1:5, 1:6, 2:7, 3:8, 4:9, 5:10, 6:11, 7:12, 8:13, 9:14, 10:15}
        self.choice = d[game.deck.board()]
        self.hands.play(self.choice)
        print(self.name, " plays ", self.choice)

class DQN_LearningPlayer(Player):
    def play(self,game,input):
        self.choice = input
        return self.hands.play(self.choice) 

class Hands:
    def __init__(self):
        self.hands = [i for i in range(1,16)]

    def play(self,num):
        try:
            self.hands.remove(num)
        except ValueError:
            #print("no hands:",num)
            return False
        return True

    def show(self):
        return self.hands

    def print(self):
        print(self.hands)

## keras-rlをインストール

In [0]:
!pip install keras-rl
!pip install gym

    100% |████████████████████████████████| 40kB 2.1MB/s 
  Running setup.py bdist_wheel for keras-rl ... - \ done
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
    100% |████████████████████████████████| 1.5MB 17.3MB/s 
    100% |████████████████████████████████| 1.0MB 22.3MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/6c/3a/0e/b86dee98876bb56cdb482cc1f72201035e46d1baf69d10d028
Successfully built gym


## Envクラスの準備

In [0]:
#from rl.core import Env
import sys

import gym
import numpy as np
import gym.spaces

class MyEnv(gym.Env):
    """すべてのエージェントによって使用される抽象環境クラス。 このクラスは厳密に
    OpenAI Gymが使用するAPIと同じAPIを使用して、OpenAI Gymとの統合が簡単です。
    また、OpenAI Gymの実装では、このクラスは実際の実装と同じです。
    独自の環境を実装するには、次のメソッドを定義する必要があります:
    - `step`
    - `reset`
    - `render`
    - `close`
    Refer to the [Gym documentation](https://gym.openai.com/docs/#environments).
    """
    def __init__(self):
        super().__init__()
        # action_space, observation_space, reward_range を設定する
        self.action_space = gym.spaces.Discrete(15)  # カード15枚
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(6,15)) # 残り手札*4 + 残り山札 + 場
        self.reward_range = [-1., 1.]
        self.reset()
        self.ren_memory = 0
    
    def step(self, action):
        """環境のダイナミクスの１つのタイムステップを実行します．
        アクションを引数にとり，(observation, reward, done, info)のタプルを戻り値にします.
        # Arguments
            action (object): 環境から提供されている１つのアクション．
        # Returns
            observation (object): 現在の環境のエージェントから見た観測値．
            reward (float) : 前回の行動の後に返された報酬の額．
            done (boolean): エピソードが終了したかどうか, そのようなケースではstep()呼び出しは未定義の結果が返されるでしょう.
            info (dict): 補助的な診断情報が含まれています（デバッグや学習に役立ちます）。
        """
        reward,obs,done = self.g.step(action)
        observation = np.zeros((6,15))
        deck = obs[0]
        dic = {-5:0,-4:1,-3:2,-2:3,-1:4,1:5,2:6,3:7,4:8,5:9,6:10,7:11,8:12,9:13,10:14}
        for card in deck:
            observation[0,dic[card]] = 1
        board = obs[1]
        observation[1,dic[board]] = 1
        
        a = obs[2]
        for card in a:
            observation[2,card-1] = 1
        b = obs[3]
        for card in b:
            observation[3,card-1] = 1
        c = obs[4]
        for card in c:
            observation[4,card-1] = 1
        d = obs[5]
        for card in d:
            observation[5,card-1] = 1
        
        return observation, reward, done, {}

    def reset(self):
        """
        Resets the state of the environment and returns an initial observation.
        # Returns
            observation (object): The initial observation of the space. Initial reward is assumed to be 0.
        """
        self.g = Game()
        self.done = False
        self.steps = 0
        self.ren_memory = 0
        obs = self.g.get_observation()
        
        observation = np.zeros((6,15))
        deck = obs[0]
        dic = {-5:0,-4:1,-3:2,-2:3,-1:4,1:5,2:6,3:7,4:8,5:9,6:10,7:11,8:12,9:13,10:14}
        for card in deck:
            observation[0,dic[card]] = 1
        board = obs[1]
        observation[1,dic[board]] = 1
        
        a = obs[2]
        for card in a:
            observation[2,card-1] = 1
        b = obs[3]
        for card in b:
            observation[3,card-1] = 1
        c = obs[4]
        for card in c:
            observation[4,card-1] = 1
        d = obs[5]
        for card in d:
            observation[5,card-1] = 1
        return observation

    def render(self, mode='human', close=False):
        """Renders the environment.
        The set of supported modes varies per environment. (And some
        environments do not support rendering at all.)
        # Arguments
            mode (str): The mode to render with.
            close (bool): Close all open renderings.
        """
        game = self.g
        if self.ren_memory == game.turn_count:
            return
        self.ren_memory = game.turn_count
        print("turn: ",game.turn_count)
        for p in game.players:
            p.print_choice()
        print("win: ",game.winner_name)
        game.show_score()
        print("\nboard:",game.deck.brd) #board
        
        

    def close(self):
        pass

    def seed(self, seed=None):
        pass

    def configure(self, *args, **kwargs):
        pass
        
    



In [0]:
from gym.envs.registration import register

register(
    id='myenv-v0',
    entry_point='myenv.env:MyEnv'
)

In [0]:
env = MyEnv()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## Processorクラスの定義

In [0]:
from rl.core import Processor

# 取りうる”打ち手”のアクション数と値の定義
nb_actions = 15


class MyProcessor(Processor):

    # Duel-DQNの出力と、Gym環境の入力の違いを吸収
    def process_action(self, action):
        return action+1

    # Gym環境の報酬の出力と、Duel-DQNの報酬の入力との違いを吸収
    def process_reward(self, reward):
        if reward > 0:
            return reward
        elif reward < 0:
            return -1
        else :
            return 0

processor = MyProcessor()

## 学習ネットワークの定義

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten


model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(nb_actions, activation="linear"))

## 学習

In [0]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam


# Duel-DQNアルゴリズム関連の幾つかの設定
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()

# Duel-DQNのAgentクラスオブジェクトの準備 （上記processorやmodelを元に）
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               enable_dueling_network=True, dueling_type="avg", target_model_update=1e-2, policy=policy,
               processor=processor)
dqn.compile(Adam(lr=1e-3), metrics=["mae"])
print(dqn.model.summary())

# 定義課題環境に対して、アルゴリズムの学習を実行 （必要に応じて適切なCallbackも定義、設定可能）
# 上記Processorクラスの適切な設定によって、Agent-環境間の入出力を通して設計課題に対しての学習が進行
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)

# 学習後のモデルの重みの出力
dqn.save_weights("duel_dqn_{}_weights.h5f".format("Pendulum-v0"), overwrite=True)

# 学習済モデルに対して、テストを実行 （必要に応じて適切なCallbackも定義、設定可能）
dqn.test(env, nb_episodes=10, visualize=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3_input (InputLayer) (None, 1, 6, 15)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 90)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               9100      
_________________________________________________________________
activation_6 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
activation_7 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10100     
__________

/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 47s 5ms/step - reward: -0.1410
1410 episodes - episode_reward: -1.000 [-1.000, -1.000] - loss: 0.070 - mean_absolute_error: 1.648 - mean_q: 2.280

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: -0.1108
1111 episodes - episode_reward: -0.997 [-1.000, 0.000] - loss: 0.046 - mean_absolute_error: 1.877 - mean_q: 2.533

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: -0.1395
1395 episodes - episode_reward: -1.000 [-1.000, -1.000] - loss: 0.004 - mean_absolute_error: 0.453 - mean_q: 0.194

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: -0.1519
1520 episodes - episode_reward: -0.999 [-1.000, 0.000] - loss: 0.001 - mean_absolute_error: 0.550 - mean_q: -0.439

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 45s 4ms/step - reward: -0.1330
1331 ep

## 追加学習

In [0]:
# 定義課題環境に対して、アルゴリズムの学習を実行 （必要に応じて適切なCallbackも定義、設定可能）
# 上記Processorクラスの適切な設定によって、Agent-環境間の入出力を通して設計課題に対しての学習が進行
dqn.fit(env, nb_steps=300000, visualize=False, verbose=1)

# 学習後のモデルの重みの出力
dqn.save_weights("duel_dqn_{}_weights.h5f".format("Pendulum-v0"), overwrite=True)

# 学習済モデルに対して、テストを実行 （必要に応じて適切なCallbackも定義、設定可能）
dqn.test(env, nb_episodes=10, visualize=True)

Training for 300000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: -0.1189
1192 episodes - episode_reward: -0.997 [-1.000, 0.000] - loss: 0.002 - mean_absolute_error: 0.523 - mean_q: 0.378

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: -0.1318
1319 episodes - episode_reward: -0.999 [-1.000, 0.000] - loss: 0.001 - mean_absolute_error: 0.361 - mean_q: 0.051

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: -0.1369
1371 episodes - episode_reward: -0.999 [-1.000, 1.000] - loss: 0.001 - mean_absolute_error: 0.343 - mean_q: -0.098

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: -0.1401
1404 episodes - episode_reward: -0.998 [-1.000, 1.000] - loss: 0.001 - mean_absolute_error: 0.358 - mean_q: -0.130

Interval 5 (40000 steps performed)
10000/10000 [============

In [0]:
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)

Training for 100000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 42s 4ms/step - reward: -0.1084
1103 episodes - episode_reward: -0.983 [-1.000, 1.000] - loss: 0.001 - mean_absolute_error: 0.698 - mean_q: 0.670

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 42s 4ms/step - reward: -0.1079
1097 episodes - episode_reward: -0.984 [-1.000, 1.000] - loss: 0.001 - mean_absolute_error: 0.762 - mean_q: 0.778

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 41s 4ms/step - reward: -0.0982
1022 episodes - episode_reward: -0.961 [-1.000, 1.000] - loss: 0.001 - mean_absolute_error: 0.816 - mean_q: 0.864

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 41s 4ms/step - reward: -0.1017
1045 episodes - episode_reward: -0.973 [-1.000, 1.000] - loss: 0.001 - mean_absolute_error: 0.825 - mean_q: 0.877

Interval 5 (40000 steps performed)
10000/10000 [==============